In [5]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("dataset\original\Chickenpox\CHP_01_01.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 809ms/step
Class: ChickenPox
Confidence Score: 0.9929972


In [4]:
from keras.preprocessing.image import ImageDataGenerator

# Path to your dataset
test_data_dir = r'C:\Users\aashi\Downloads\archive (11)\Original Images\Original Images\FOLDS\fold1\Test'

# Create a data generator for loading and preprocessing images
test_datagen = ImageDataGenerator(rescale=1./127.5, preprocessing_function=lambda x: x - 1)

# Load the test images from the directory
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Same size as used for training
    batch_size=32,
    class_mode='categorical',  # Assumes you have multiple classes
    shuffle=False  # No need to shuffle for evaluation
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)

# Print accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 74 images belonging to 6 classes.


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [1]:
import torch
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM



tokenizer = AutoTokenizer.from_pretrained("jianghc/medical_chatbot")
model = AutoModelForCausalLM.from_pretrained("jianghc/medical_chatbot")

# path = "jianghc/medical_chatbot"
device = "cuda" if torch.cuda.is_available() else "cpu"
# tokenizer = GPT2Tokenizer.from_pretrained(path)
# model = GPT2LMHeadModel.from_pretrained(path).to(device)

prompt_input = (
    "The conversation between human and AI assistant.\n"
    "[|Human|] {input}\n"
    "[|AI|]"
)
sentence = prompt_input.format_map({'input': "what is parkinson's disease?"})
inputs = tokenizer(sentence, return_tensors="pt").to(device)

with torch.no_grad():
    beam_output = model.generate(**inputs,
                                min_new_tokens=1, 
                                max_length=512,
                                num_beams=3,
                                repetition_penalty=1.2,
                                early_stopping=True,
                                eos_token_id=198 
                                )
    print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


tokenizer_config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

c:\Users\aashi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aashi\.cache\huggingface\hub\models--jianghc--medical_chatbot. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/584k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

c:\Users\aashi\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/10.4G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
sentence = prompt_input.format_map({'input': "what is parkinson's disease?"})
inputs = tokenizer(sentence, return_tensors="pt").to(device)

with torch.no_grad():
    beam_output = model.generate(**inputs,
                                min_new_tokens=1, 
                                max_length=512,
                                num_beams=3,
                                repetition_penalty=1.2,
                                early_stopping=True,
                                eos_token_id=198 
                                )
    print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
model_name = "ruslanmv/Medical-Llama3-8B"
device_map = 'auto'
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16,)
model = AutoModelForCausalLM.from_pretrained( model_name,quantization_config=bnb_config, trust_remote_code=True,use_cache=False,device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def askme(question):
    sys_message = '''
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''   
    # Create messages structured for the chat template
    messages = [{"role": "system", "content": sys_message}, {"role": "user", "content": question}]
    
    # Applying chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    
    # Extract and return the generated text, removing the prompt
    response_text = tokenizer.batch_decode(outputs)[0].strip()
    answer = response_text.split('<|im_start|>assistant')[-1].strip()
    return answer
# Example usage
# - Context: First describe your problem.
# - Question: Then make the question.

question = '''I'm a 35-year-old male and for the past few months, I've been experiencing fatigue, 
increased sensitivity to cold, and dry, itchy skin. 
Could these symptoms be related to hypothyroidism? 
If so, what steps should I take to get a proper diagnosis and discuss treatment options?'''

print(askme(question))


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "ruslanmv/Medical-Llama3-8B"
device = 'cpu'

# Load model and tokenizer without quantization
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, use_cache=False)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def askme(question):
    sys_message = ''' 
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''   
    # Create messages structured for the chat template
    messages = [{"role": "system", "content": sys_message}, {"role": "user", "content": question}]
    
    # Applying chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    
    # Extract and return the generated text, removing the prompt
    response_text = tokenizer.batch_decode(outputs)[0].strip()
    answer = response_text.split('<|im_start|>assistant')[-1].strip()
    return answer

# Example usage
question = '''I'm a 35-year-old male and for the past few months, I've been experiencing fatigue, 
increased sensitivity to cold, and dry, itchy skin. 
Could these symptoms be related to hypothyroidism? 
If so, what steps should I take to get a proper diagnosis and discuss treatment options?'''

print(askme(question))


: 

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = './saved_model'
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


OSError: Incorrect path_or_model_id: './saved_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

In [2]:
def askme(question):
    sys_message = ''' 
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''   
    # Create messages structured for the chat template
    messages = [{"role": "system", "content": sys_message}, {"role": "user", "content": question}]
    
    # Applying chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    
    # Extract and return the generated text, removing the prompt
    response_text = tokenizer.batch_decode(outputs)[0].strip()
    answer = response_text.split('<|im_start|>assistant')[-1].strip()
    return answer

# Example usage
question = '''I'm a 35-year-old male and for the past few months, I've been experiencing fatigue, 
increased sensitivity to cold, and dry, itchy skin. 
Could these symptoms be related to hypothyroidism? 
If so, what steps should I take to get a proper diagnosis and discuss treatment options?'''

print(askme(question))


NameError: name 'tokenizer' is not defined

In [1]:
import shutil
cache_dir = '~/.cache/huggingface/transformers'
shutil.rmtree(cache_dir, ignore_errors=True)


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "ruslanmv/Medical-Llama3-8B"
device = 'cpu'

print("Starting script...")

try:
    print("Loading model...")
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, use_cache=False)
    print("Model loaded.")
     
    print("Saving model...")
    model.save_pretrained('./saved_model')
except Exception as e:
    print("Error during model/tokenizer loading or saving:", e)


Starting script...
Loading model...


: 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "ruslanmv/Medical-Llama3-8B"
device = 'cpu'

print("Starting script...")

try:
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer loaded.")
    
    print("Saving model and tokenizer...")
    model.save_pretrained('./saved_model')
    tokenizer.save_pretrained('./saved_model')
    print("Model and tokenizer saved.")

except Exception as e:
    print("Error during model/tokenizer loading or saving:", e)



Starting script...
Loading tokenizer...
Tokenizer loaded.
Saving model and tokenizer...
Model and tokenizer saved.


In [1]:
print("hi")

hi


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("AdaptLLM/medicine-chat")
tokenizer = AutoTokenizer.from_pretrained("AdaptLLM/medicine-chat")

# Put your input here:
user_input = '''Question: Which of the following is an example of monosomy?
Options:
- 46,XX
- 47,XXX
- 69,XYY
- 45,X

Please provide your choice first and then provide explanations if possible.'''

# Apply the prompt template and system prompt of LLaMA-2-Chat demo for chat models (NOTE: NO prompt template is required for base models!)
our_system_prompt = "\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n" # Please do NOT change this
prompt = f"<s>[INST] <<SYS>>{our_system_prompt}<</SYS>>\n\n{user_input} [/INST]"

# # NOTE:
# # If you want to apply your own system prompt, please integrate it into the instruction part following our system prompt like this:
# your_system_prompt = "Please, answer this question faithfully."
# prompt = f"<s>[INST] <<SYS>>{our_system_prompt}<</SYS>>\n\n{your_system_prompt}\n{user_input} [/INST]"

inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model.device)
outputs = model.generate(input_ids=inputs, max_length=4096)[0]

answer_start = int(inputs.shape[-1])
pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)

print(pred)


c:\Users\aashi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:991: UserWarning: Not enough free disk space to download the file. The expected file size is: 9877.98 MB. The target location C:\Users\aashi\.cache\huggingface\hub\models--AdaptLLM--medicine-chat\blobs only has 2656.42 MB free disk space.
  warnings.warn(


model-00001-of-00003.safetensors:   0%|          | 21.0M/9.88G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
  
print("Saving model and tokenizer...")
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')
print("Model and tokenizer saved.")